In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from tensorflow.keras import regularizers
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout, Rescaling
from keras.optimizers import Adam
from keras import regularizers
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.vgg16 import  preprocess_input
from tensorflow.keras import layers, models, applications
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import VGG16


import cv2
import pandas as pd
import numpy as np

from IPython.display import display
import logging

physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

print("##############################################")
print("##############################################")

print("##############################################")
print("##############################################")

# Suppress TensorFlow logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Define the directory and image properties
data_dir = "./trainingsdaten"
img_height = 768
img_width = 1024
input_shape = (img_height, img_width, 3)
epochs = 15
# Load the training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels='inferred',  # Labels are generated from directory structure
    label_mode='int',  # Ensure labels are integers
    color_mode='rgb',
    validation_split=0.2,  # 20% of the images get used for validation
    subset="training",
    seed=123,  # For reproducing results
    image_size=(img_height, img_width),  # Using full resolution
    batch_size=2
)

# Load the validation dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='int',  # Ensure labels are integers
    color_mode='rgb',
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=2
)

test_dir = "./testdaten"
# Load the test data with file paths
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    color_mode='rgb',
    image_size=(img_height, img_width),
    batch_size=1,
    shuffle=False  # Important to not shuffle test data
)


# InceptionV3 Simple Model
base_model_inceptionv3 = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
base_model_inceptionv3.trainable = False
preprocess_input_inceptionv3 = applications.inception_v3.preprocess_input

print('##############################')
print('this is my first model with inceptionv3 that got the best result of 0.89 with config 16,32,32,32 with 0.01 regul')
print('##############################')

# InceptionV3 Complex Model with L2 Regularization
inception_16_3x32_001regul = models.Sequential([
    layers.Input(shape=input_shape),
    layers.Lambda(preprocess_input_inceptionv3),
    base_model_inceptionv3,

    # Convolutional layers with L2 regularization
    layers.Conv2D(16, 3, padding='same', activation='relu',
                  kernel_regularizer=regularizers.l2(0.01)),
    layers.MaxPooling2D(),

    layers.Conv2D(32, 3, padding='same', activation='relu',
                  kernel_regularizer=regularizers.l2(0.01)),
    layers.MaxPooling2D(),

    layers.Conv2D(32, 3, padding='same', activation='relu',
                  kernel_regularizer=regularizers.l2(0.01)),
    layers.MaxPooling2D(),

    layers.Conv2D(32, 3, padding='same', activation='relu',
                  kernel_regularizer=regularizers.l2(0.01)),
    layers.MaxPooling2D(),

    layers.Flatten(),

    # Dense layer with L2 regularization
    layers.Dense(128, activation='sigmoid',
                 kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(2)
])

# Compile the model
inception_16_3x32_001regul.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

# Train the model
print("Training Complex Model with InceptionV3 and Regularization:")
inception_16_3x32_001regul_history = inception_16_3x32_001regul.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    verbose=2
)

# Testing phase remains the same
print('Test the model I just trained')

# Initialize lists to store results
image_names = []
actual_labels = []
predicted_labels = []
predicted_probs = []

class_names = test_ds.class_names

# Iterate over the test dataset
for images, labels in test_ds:
    # Get image name
    image_name = test_ds.file_paths[len(image_names)]

    # Make predictions
    preds = inception_16_3x32_001regul.predict(images)
    pred_probs = tf.nn.softmax(preds, axis=-1)

    # Get predicted label
    predicted_label = np.argmax(pred_probs, axis=-1)

    # Store results
    image_names.append(image_name)
    actual_labels.append(class_names[labels[0]])  # labels is already a numpy array
    predicted_labels.append(class_names[predicted_label[0]])  # predicted_label is already a numpy array
    predicted_probs.append(pred_probs.numpy()[0].tolist())

# Calculate misclassification rate
misclassified_count = sum([1 for actual, predicted in zip(actual_labels, predicted_labels) if actual != predicted])
total_samples = len(image_names)
misclassification_rate = misclassified_count / total_samples

# Create DataFrame
results_df3 = pd.DataFrame({
    'Image Name': image_names,
    'Actual Label': actual_labels,
    'Predicted Label': predicted_labels,
    'Predicted Probability': predicted_probs
})

print(misclassified_count, misclassification_rate)
display(results_df3.to_string())

print('##############################')
print('this is my first model with inceptionv3 that got the best result of 0.89 with config 128_64_32_16')
print('##############################')

# InceptionV3 Complex Model
inception_128_64_32_16 = models.Sequential([
    layers.Input(shape=input_shape),
    layers.Lambda(preprocess_input_inceptionv3),
    base_model_inceptionv3,
    layers.Conv2D(128, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='sigmoid'),
    layers.Dense(2)
])

inception_128_64_32_16.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

print("Training Complex Model with InceptionV3:")
inception_128_64_32_16_history = inception_128_64_32_16.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    verbose=2
)

print('Test the model I just trained')
# Initialize lists to store results
image_names = []
actual_labels = []
predicted_labels = []
predicted_probs = []

class_names = test_ds.class_names

# Iterate over the test dataset
for images, labels in test_ds:
    # Get image name
    image_name = test_ds.file_paths[len(image_names)]

    # Make predictions
    preds = inception_128_64_32_16.predict(images)
    pred_probs = tf.nn.softmax(preds, axis=-1)

    # Get predicted label
    predicted_label = np.argmax(pred_probs, axis=-1)

    # Store results
    image_names.append(image_name)
    actual_labels.append(class_names[labels[0]])  # labels is already a numpy array
    predicted_labels.append(class_names[predicted_label[0]])  # predicted_label is already a numpy array
    predicted_probs.append(pred_probs.numpy()[0].tolist())

# Calculate misclassification rate
misclassified_count = sum([1 for actual, predicted in zip(actual_labels, predicted_labels) if actual != predicted])
total_samples = len(image_names)
misclassification_rate = misclassified_count / total_samples

# Create DataFrame
results_df3 = pd.DataFrame({
    'Image Name': image_names,
    'Actual Label': actual_labels,
    'Predicted Label': predicted_labels,
    'Predicted Probability': predicted_probs
})

print(misclassified_count, misclassification_rate)
display(results_df3.to_string())

print('##############################')
print('this is my first model with inceptionv3 that got the best result of 0.89 with config 16,32,32,32 with no conv2d')
# print('ORIGNIAL MODEL')
print('##############################')

# InceptionV3 Complex Model
inception_16_3x32_no_conv = models.Sequential([
    layers.Input(shape=input_shape),
    layers.Lambda(preprocess_input_inceptionv3),
    base_model_inceptionv3,
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.Flatten(),
    layers.Dense(128, activation='sigmoid'),
    layers.Dense(2)
])

inception_16_3x32_no_conv.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

print("Training Complex Model with InceptionV3:")
inception_16_3x32_no_conv_history = inception_16_3x32_no_conv.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    verbose=2
)

print('Test the model I just trained')
# Initialize lists to store results
image_names = []
actual_labels = []
predicted_labels = []
predicted_probs = []

class_names = test_ds.class_names

# Iterate over the test dataset
for images, labels in test_ds:
    # Get image name
    image_name = test_ds.file_paths[len(image_names)]

    # Make predictions
    preds = inception_16_3x32_no_conv.predict(images)
    pred_probs = tf.nn.softmax(preds, axis=-1)

    # Get predicted label
    predicted_label = np.argmax(pred_probs, axis=-1)

    # Store results
    image_names.append(image_name)
    actual_labels.append(class_names[labels[0]])  # labels is already a numpy array
    predicted_labels.append(class_names[predicted_label[0]])  # predicted_label is already a numpy array
    predicted_probs.append(pred_probs.numpy()[0].tolist())

# Calculate misclassification rate
misclassified_count = sum([1 for actual, predicted in zip(actual_labels, predicted_labels) if actual != predicted])
total_samples = len(image_names)
misclassification_rate = misclassified_count / total_samples

# Create DataFrame
results_df3 = pd.DataFrame({
    'Image Name': image_names,
    'Actual Label': actual_labels,
    'Predicted Label': predicted_labels,
    'Predicted Probability': predicted_probs
})

print(misclassified_count, misclassification_rate)
display(results_df3.to_string())

print('##############################')
print('this is my first model with inceptionv3 that got the best result of 0.89 with config 16,32,32,32 with 1 conv2d')
# print('ORIGNIAL MODEL')
print('##############################')

# InceptionV3 Complex Model
inception_16_3x32_1_conv = models.Sequential([
    layers.Input(shape=input_shape),
    layers.Lambda(preprocess_input_inceptionv3),
    base_model_inceptionv3,
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='sigmoid'),
    layers.Dense(2)
])

inception_16_3x32_1_conv.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

print("Training Complex Model with InceptionV3:")
inception_16_3x32_1_conv_history = inception_16_3x32_1_conv.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    verbose=2
)

print('Test the model I just trained')
# Initialize lists to store results
image_names = []
actual_labels = []
predicted_labels = []
predicted_probs = []

class_names = test_ds.class_names

# Iterate over the test dataset
for images, labels in test_ds:
    # Get image name
    image_name = test_ds.file_paths[len(image_names)]

    # Make predictions
    preds = inception_16_3x32_1_conv.predict(images)
    pred_probs = tf.nn.softmax(preds, axis=-1)

    # Get predicted label
    predicted_label = np.argmax(pred_probs, axis=-1)

    # Store results
    image_names.append(image_name)
    actual_labels.append(class_names[labels[0]])  # labels is already a numpy array
    predicted_labels.append(class_names[predicted_label[0]])  # predicted_label is already a numpy array
    predicted_probs.append(pred_probs.numpy()[0].tolist())

# Calculate misclassification rate
misclassified_count = sum([1 for actual, predicted in zip(actual_labels, predicted_labels) if actual != predicted])
total_samples = len(image_names)
misclassification_rate = misclassified_count / total_samples

# Create DataFrame
results_df3 = pd.DataFrame({
    'Image Name': image_names,
    'Actual Label': actual_labels,
    'Predicted Label': predicted_labels,
    'Predicted Probability': predicted_probs
})

print(misclassified_count, misclassification_rate)
display(results_df3.to_string())

print('##############################')
print('this is my first model with inceptionv3 that got the best result of 0.89 with config 16,32,32,32 with 2 conv2d')
print('##############################')

# InceptionV3 Complex Model
inception_16_3x32_2_conv = models.Sequential([
    layers.Input(shape=input_shape),
    layers.Lambda(preprocess_input_inceptionv3),
    base_model_inceptionv3,
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='sigmoid'),
    layers.Dense(2)
])

inception_16_3x32_2_conv.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

print("Training Complex Model with InceptionV3:")
inception_16_3x32_2_conv_history = inception_16_3x32_2_conv.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    verbose=2
)

print('Test the model I just trained')
# Initialize lists to store results
image_names = []
actual_labels = []
predicted_labels = []
predicted_probs = []

class_names = test_ds.class_names

# Iterate over the test dataset
for images, labels in test_ds:
    # Get image name
    image_name = test_ds.file_paths[len(image_names)]

    # Make predictions
    preds = inception_16_3x32_2_conv.predict(images)
    pred_probs = tf.nn.softmax(preds, axis=-1)

    # Get predicted label
    predicted_label = np.argmax(pred_probs, axis=-1)

    # Store results
    image_names.append(image_name)
    actual_labels.append(class_names[labels[0]])  # labels is already a numpy array
    predicted_labels.append(class_names[predicted_label[0]])  # predicted_label is already a numpy array
    predicted_probs.append(pred_probs.numpy()[0].tolist())

# Calculate misclassification rate
misclassified_count = sum([1 for actual, predicted in zip(actual_labels, predicted_labels) if actual != predicted])
total_samples = len(image_names)
misclassification_rate = misclassified_count / total_samples

# Create DataFrame
results_df3 = pd.DataFrame({
    'Image Name': image_names,
    'Actual Label': actual_labels,
    'Predicted Label': predicted_labels,
    'Predicted Probability': predicted_probs
})

print(misclassified_count, misclassification_rate)
display(results_df3.to_string())

# InceptionV3 Simple Model
base_model_inceptionv3 = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
base_model_inceptionv3.trainable = False
preprocess_input_inceptionv3 = applications.inception_v3.preprocess_input

print('##############################')
print('this is my first model with inceptionv3 that got the best result of 0.89 with config 16,32x4')
print('##############################')

# InceptionV3 Complex Model
inception_16_4x32 = models.Sequential([
    layers.Input(shape=input_shape),
    layers.Lambda(preprocess_input_inceptionv3),
    base_model_inceptionv3,
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='sigmoid'),
    layers.Dense(2)
])

inception_16_4x32.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

print("Training Complex Model with InceptionV3:")
inception_16_4x32_history = inception_16_4x32.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    verbose=2
)

print('Test the model I just trained')
# Initialize lists to store results
image_names = []
actual_labels = []
predicted_labels = []
predicted_probs = []

class_names = test_ds.class_names

# Iterate over the test dataset
for images, labels in test_ds:
    # Get image name
    image_name = test_ds.file_paths[len(image_names)]

    # Make predictions
    preds = inception_16_4x32.predict(images)
    pred_probs = tf.nn.softmax(preds, axis=-1)

    # Get predicted label
    predicted_label = np.argmax(pred_probs, axis=-1)

    # Store results
    image_names.append(image_name)
    actual_labels.append(class_names[labels[0]])  # labels is already a numpy array
    predicted_labels.append(class_names[predicted_label[0]])  # predicted_label is already a numpy array
    predicted_probs.append(pred_probs.numpy()[0].tolist())

# Calculate misclassification rate
misclassified_count = sum([1 for actual, predicted in zip(actual_labels, predicted_labels) if actual != predicted])
total_samples = len(image_names)
misclassification_rate = misclassified_count / total_samples

# Create DataFrame
results_df3 = pd.DataFrame({
    'Image Name': image_names,
    'Actual Label': actual_labels,
    'Predicted Label': predicted_labels,
    'Predicted Probability': predicted_probs
})

print(misclassified_count, misclassification_rate)
display(results_df3.to_string())

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
##############################################
##############################################
Testing regulizers
##############################################
##############################################
Found 38091 files belonging to 2 classes.
Using 30473 files for training.
Found 38091 files belonging to 2 classes.
Using 7618 files for validation.
Found 868 files belonging to 2 classes.
##############################
this is my first model with inceptionv3 that got the best result of 0.89 with config 16,32,32,32 with 0.01 regul
##############################
Training Complex Model with InceptionV3 and Regularization:
Epoch 1/15
15237/15237 - 367s - 24ms/step - accuracy: 0.8239 - loss: 0.5256 - val_accuracy: 0.8523 - val_loss: 0.4556
Epoch 2/15
15237/15237 - 352s - 23ms/step - accuracy: 0.8312 - loss: 0.4794 - val_accuracy: 0.8514 - val_loss: 0.4531
Epoch 3/15
15237/15237 - 363s - 24ms/step - accuracy: 0.8328 - los

2024-10-20 22:42:03.498651: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


'                                   Image Name Actual Label Predicted Label                       Predicted Probability\n0    ./testdaten/NOK/PW23-0031-01_1_10143.png          NOK             NOK     [0.771625816822052, 0.2283741682767868]\n1    ./testdaten/NOK/PW23-0031-01_1_10287.png          NOK             NOK   [0.7403263449668884, 0.25967368483543396]\n2    ./testdaten/NOK/PW23-0031-01_1_10449.png          NOK             NOK   [0.7820054292678833, 0.21799463033676147]\n3     ./testdaten/NOK/PW23-0031-01_1_1053.png          NOK             NOK    [0.6907731294631958, 0.3092268705368042]\n4    ./testdaten/NOK/PW23-0031-01_1_10629.png          NOK             NOK    [0.6328640580177307, 0.3671359121799469]\n5    ./testdaten/NOK/PW23-0031-01_1_10683.png          NOK             NOK   [0.8893872499465942, 0.11061274260282516]\n6    ./testdaten/NOK/PW23-0031-01_1_10701.png          NOK             NOK    [0.6953386664390564, 0.3046613335609436]\n7    ./testdaten/NOK/PW23-0031-01_1_108

##############################
this is my first model with inceptionv3 that got the best result of 0.89 with config 128_64_32_16
##############################
Training Complex Model with InceptionV3:
Epoch 1/15
15237/15237 - 383s - 25ms/step - accuracy: 0.6283 - loss: 0.6145 - val_accuracy: 0.5102 - val_loss: 0.6954
Epoch 2/15
15237/15237 - 370s - 24ms/step - accuracy: 0.5040 - loss: 0.7043 - val_accuracy: 0.5100 - val_loss: 0.6935
Epoch 3/15
15237/15237 - 370s - 24ms/step - accuracy: 0.5014 - loss: 0.7045 - val_accuracy: 0.4899 - val_loss: 0.6934
Epoch 4/15
15237/15237 - 370s - 24ms/step - accuracy: 0.5020 - loss: 0.7046 - val_accuracy: 0.5100 - val_loss: 0.6954
Epoch 5/15
15237/15237 - 371s - 24ms/step - accuracy: 0.5012 - loss: 0.7051 - val_accuracy: 0.5102 - val_loss: 0.7041
Epoch 6/15
15237/15237 - 370s - 24ms/step - accuracy: 0.5000 - loss: 0.7052 - val_accuracy: 0.5100 - val_loss: 0.6929
Epoch 7/15
15237/15237 - 370s - 24ms/step - accuracy: 0.5038 - loss: 0.7050 - val_accuracy:

2024-10-21 00:15:38.755370: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

2024-10-21 04:47:09.093389: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


'                                   Image Name Actual Label Predicted Label                       Predicted Probability\n0    ./testdaten/NOK/PW23-0031-01_1_10143.png          NOK              OK   [0.47584065794944763, 0.5241593718528748]\n1    ./testdaten/NOK/PW23-0031-01_1_10287.png          NOK              OK   [0.47584065794944763, 0.5241593718528748]\n2    ./testdaten/NOK/PW23-0031-01_1_10449.png          NOK              OK   [0.47584065794944763, 0.5241593718528748]\n3     ./testdaten/NOK/PW23-0031-01_1_1053.png          NOK              OK   [0.47584065794944763, 0.5241593718528748]\n4    ./testdaten/NOK/PW23-0031-01_1_10629.png          NOK              OK   [0.47584065794944763, 0.5241593718528748]\n5    ./testdaten/NOK/PW23-0031-01_1_10683.png          NOK              OK   [0.47584065794944763, 0.5241593718528748]\n6    ./testdaten/NOK/PW23-0031-01_1_10701.png          NOK              OK   [0.47584065794944763, 0.5241593718528748]\n7    ./testdaten/NOK/PW23-0031-01_1_108

##############################
this is my first model with inceptionv3 that got the best result of 0.89 with config 16,32x4
##############################
Training Complex Model with InceptionV3:
Epoch 1/15
15237/15237 - 369s - 24ms/step - accuracy: 0.8437 - loss: 0.3813 - val_accuracy: 0.8637 - val_loss: 0.3375
Epoch 2/15
15237/15237 - 356s - 23ms/step - accuracy: 0.8592 - loss: 0.3418 - val_accuracy: 0.8564 - val_loss: 0.3435
Epoch 3/15
15237/15237 - 356s - 23ms/step - accuracy: 0.8633 - loss: 0.3283 - val_accuracy: 0.8721 - val_loss: 0.3235
Epoch 4/15
15237/15237 - 356s - 23ms/step - accuracy: 0.8689 - loss: 0.3165 - val_accuracy: 0.8601 - val_loss: 0.3551
Epoch 5/15
15237/15237 - 356s - 23ms/step - accuracy: 0.8726 - loss: 0.3126 - val_accuracy: 0.8687 - val_loss: 0.3296
Epoch 6/15
15237/15237 - 356s - 23ms/step - accuracy: 0.8700 - loss: 0.3217 - val_accuracy: 0.8693 - val_loss: 0.3203
Epoch 7/15
15237/15237 - 356s - 23ms/step - accuracy: 0.8615 - loss: 0.3397 - val_accuracy: 0.84

2024-10-21 06:17:11.541041: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


'                                   Image Name Actual Label Predicted Label                       Predicted Probability\n0    ./testdaten/NOK/PW23-0031-01_1_10143.png          NOK             NOK    [0.8922002911567688, 0.1077997013926506]\n1    ./testdaten/NOK/PW23-0031-01_1_10287.png          NOK             NOK   [0.5625962615013123, 0.43740376830101013]\n2    ./testdaten/NOK/PW23-0031-01_1_10449.png          NOK             NOK   [0.5625962615013123, 0.43740376830101013]\n3     ./testdaten/NOK/PW23-0031-01_1_1053.png          NOK              OK    [0.3939231336116791, 0.6060768365859985]\n4    ./testdaten/NOK/PW23-0031-01_1_10629.png          NOK             NOK   [0.5625962615013123, 0.43740376830101013]\n5    ./testdaten/NOK/PW23-0031-01_1_10683.png          NOK             NOK   [0.8622646331787109, 0.13773536682128906]\n6    ./testdaten/NOK/PW23-0031-01_1_10701.png          NOK              OK    [0.3689247667789459, 0.6310752630233765]\n7    ./testdaten/NOK/PW23-0031-01_1_108

In [25]:
# Initialize lists to store results
image_names = []
actual_labels = []
predicted_labels = []
predicted_probs = []
unclassified_images = []  # List to store names of unclassified images

class_names = test_ds.class_names

# Threshold for classification
prob_threshold = 0.575

# Iterate over the test dataset
for images, labels in test_ds:
    # Get image name
    image_name = test_ds.file_paths[len(image_names)]

    # Make predictions
    preds = inception_16_3x32_no_conv.predict(images)
    pred_probs = tf.nn.softmax(preds, axis=-1)

    # Get predicted label and probability for the most likely class
    predicted_label = np.argmax(pred_probs, axis=-1)
    max_prob = np.max(pred_probs, axis=-1)

    # Apply threshold: Only classify if max_prob > 0.6
    if max_prob[0] > prob_threshold:
        # Store results if classification is made
        image_names.append(image_name)
        actual_labels.append(class_names[labels[0]])  # labels is already a numpy array
        predicted_labels.append(class_names[predicted_label[0]])  # predicted_label is already a numpy array
        predicted_probs.append(pred_probs.numpy()[0].tolist())
    else:
        # Store unclassified image name if max probability is below threshold
        unclassified_images.append(image_name)

# Calculate misclassification rate
misclassified_count = sum([1 for actual, predicted in zip(actual_labels, predicted_labels) if actual != predicted])
total_samples = len(image_names)  # Now only classified samples are considered
misclassification_rate = misclassified_count / total_samples if total_samples > 0 else 0

# Create DataFrame
results_df3 = pd.DataFrame({
    'Image Name': image_names,
    'Actual Label': actual_labels,
    'Predicted Label': predicted_labels,
    'Predicted Probability': predicted_probs
})

# Output results
print("Misclassified count:", misclassified_count)
print("Misclassification rate:", misclassification_rate)
print(f"Number of unclassified images (below {prob_threshold} probability): {len(unclassified_images)}")

display(results_df3.to_string())

# Optionally, you can save the unclassified images list for further inspection
unclassified_df = pd.DataFrame(unclassified_images)
display(unclassified_df.to_string())


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━

2024-10-21 07:42:34.267530: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


'Empty DataFrame\nColumns: [Image Name, Actual Label, Predicted Label, Predicted Probability]\nIndex: []'

'                                            0\n0    ./testdaten/NOK/PW23-0031-01_1_10143.png\n1    ./testdaten/NOK/PW23-0031-01_1_10143.png\n2    ./testdaten/NOK/PW23-0031-01_1_10143.png\n3    ./testdaten/NOK/PW23-0031-01_1_10143.png\n4    ./testdaten/NOK/PW23-0031-01_1_10143.png\n5    ./testdaten/NOK/PW23-0031-01_1_10143.png\n6    ./testdaten/NOK/PW23-0031-01_1_10143.png\n7    ./testdaten/NOK/PW23-0031-01_1_10143.png\n8    ./testdaten/NOK/PW23-0031-01_1_10143.png\n9    ./testdaten/NOK/PW23-0031-01_1_10143.png\n10   ./testdaten/NOK/PW23-0031-01_1_10143.png\n11   ./testdaten/NOK/PW23-0031-01_1_10143.png\n12   ./testdaten/NOK/PW23-0031-01_1_10143.png\n13   ./testdaten/NOK/PW23-0031-01_1_10143.png\n14   ./testdaten/NOK/PW23-0031-01_1_10143.png\n15   ./testdaten/NOK/PW23-0031-01_1_10143.png\n16   ./testdaten/NOK/PW23-0031-01_1_10143.png\n17   ./testdaten/NOK/PW23-0031-01_1_10143.png\n18   ./testdaten/NOK/PW23-0031-01_1_10143.png\n19   ./testdaten/NOK/PW23-0031-01_1_10143.png\n20   ./testd

In [26]:
    print('##############################')
    print('this is my first model with inceptionv3 that got the best result of 0.89 with config 16_32_64_128')
    print('##############################')



    # InceptionV3 Complex Model
    most_complex_model_inceptionv3 = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Lambda(preprocess_input_inceptionv3),
        base_model_inceptionv3,
        layers.Conv2D(16, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(32, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation='sigmoid'),
        layers.Dense(2)
    ])

    most_complex_model_inceptionv3.compile(
        optimizer='adam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'],
    )

    print("Training Complex Model with InceptionV3:")
    most_complex_history_inceptionv3 = most_complex_model_inceptionv3.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        verbose=2
    )

    print('Test the model I just trained')
    # Initialize lists to store results
    image_names = []
    actual_labels = []
    predicted_labels = []
    predicted_probs = []

    class_names = test_ds.class_names

    # Iterate over the test dataset
    for images, labels in test_ds:
        # Get image name
        image_name = test_ds.file_paths[len(image_names)]

        # Make predictions
        preds = most_complex_model_inceptionv3.predict(images)
        pred_probs = tf.nn.softmax(preds, axis=-1)

        # Get predicted label
        predicted_label = np.argmax(pred_probs, axis=-1)

        # Store results
        image_names.append(image_name)
        actual_labels.append(class_names[labels[0]])  # labels is already a numpy array
        predicted_labels.append(class_names[predicted_label[0]])  # predicted_label is already a numpy array
        predicted_probs.append(pred_probs.numpy()[0].tolist())

    # Calculate misclassification rate
    misclassified_count = sum([1 for actual, predicted in zip(actual_labels, predicted_labels) if actual != predicted])
    total_samples = len(image_names)
    misclassification_rate = misclassified_count / total_samples

    # Create DataFrame
    results_df3 = pd.DataFrame({
        'Image Name': image_names,
        'Actual Label': actual_labels,
        'Predicted Label': predicted_labels,
        'Predicted Probability': predicted_probs
    })

    print(misclassified_count, misclassification_rate)
    display(results_df3.to_string())

##############################
this is my first model with inceptionv3 that got the best result of 0.89 with config 16_32_64_128
##############################
Training Complex Model with InceptionV3:
Epoch 1/15
15237/15237 - 373s - 25ms/step - accuracy: 0.7772 - loss: 0.4945 - val_accuracy: 0.8104 - val_loss: 0.4542
Epoch 2/15
15237/15237 - 363s - 24ms/step - accuracy: 0.5697 - loss: 0.6546 - val_accuracy: 0.5102 - val_loss: 0.6933
Epoch 3/15
15237/15237 - 366s - 24ms/step - accuracy: 0.5017 - loss: 0.7030 - val_accuracy: 0.5102 - val_loss: 0.6943
Epoch 4/15
15237/15237 - 365s - 24ms/step - accuracy: 0.5023 - loss: 0.7021 - val_accuracy: 0.5102 - val_loss: 0.6931
Epoch 5/15
15237/15237 - 361s - 24ms/step - accuracy: 0.5000 - loss: 0.7018 - val_accuracy: 0.5102 - val_loss: 0.6930
Epoch 6/15
15237/15237 - 362s - 24ms/step - accuracy: 0.5032 - loss: 0.7019 - val_accuracy: 0.4898 - val_loss: 0.6932
Epoch 7/15
15237/15237 - 362s - 24ms/step - accuracy: 0.5025 - loss: 0.7015 - val_accuracy:

2024-10-21 09:16:31.090726: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


'                                   Image Name Actual Label Predicted Label                      Predicted Probability\n0    ./testdaten/NOK/PW23-0031-01_1_10143.png          NOK             NOK  [0.5125057697296143, 0.48749426007270813]\n1    ./testdaten/NOK/PW23-0031-01_1_10287.png          NOK             NOK  [0.5125057697296143, 0.48749426007270813]\n2    ./testdaten/NOK/PW23-0031-01_1_10449.png          NOK             NOK  [0.5125057697296143, 0.48749426007270813]\n3     ./testdaten/NOK/PW23-0031-01_1_1053.png          NOK             NOK  [0.5125057697296143, 0.48749426007270813]\n4    ./testdaten/NOK/PW23-0031-01_1_10629.png          NOK             NOK  [0.5125057697296143, 0.48749426007270813]\n5    ./testdaten/NOK/PW23-0031-01_1_10683.png          NOK             NOK  [0.5125057697296143, 0.48749426007270813]\n6    ./testdaten/NOK/PW23-0031-01_1_10701.png          NOK             NOK  [0.5125057697296143, 0.48749426007270813]\n7    ./testdaten/NOK/PW23-0031-01_1_10809.png  